In [47]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [48]:
bnf_pandas = pd.read_csv("../data/indices/NIFTY BANK.csv")

In [49]:
# symbol = 'midcp'
# symbol = 'nifty'
# symbol = 'fnf'
symbol = 'bnf'
# symbol = 'sensex'
# symbol = 'bankex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or symbol == 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 8
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 8
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1000000
print(LEVERAGE_)

5


In [50]:
bnf_pandas.head()

,datetime,o,h,l,c
0,2017-01-02T09:15:00.000000,18242.3000,18248.2000,18175.9000,18181.2000
1,2017-01-02T09:16:00.000000,18181.8500,18194.7000,18179.9500,18184.4500
2,2017-01-02T09:17:00.000000,18184.9500,18189.2500,18133.8000,18133.8000
3,2017-01-02T09:18:00.000000,18135.1000,18141.5500,18118.5500,18138.9500
4,2017-01-02T09:19:00.000000,18138.9500,18142.5500,18120.4500,18124.3000


In [51]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [52]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
bnf_pandas.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)
spot_df = bnf_pandas

<class 'polars.dataframe.frame.DataFrame'>


In [53]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

In [54]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )

In [55]:
bnf2 = resample(bnf, '15m')
bnf_pandas = bnf2.to_pandas()
bnf_pandas.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)

In [56]:
def ma_crossover_logic(df, fast_ma, slow_ma):
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    
    fast_ema = df['close'].rolling(fast_ma).mean()
    slow_ema = df['close'].rolling(slow_ma).mean()
    df['fast_ma'] = fast_ema
    df['slow_ma'] = slow_ema
    
    df['signal'] = 0  # Default to no signal
    df.loc[fast_ema > slow_ema, 'signal'] = 1   # Long Signal
    df.loc[fast_ema < slow_ema, 'signal'] = -1  # Short Signal
    
    return df

In [74]:
def execute(df, spot_df, sl_pct, n, portfolio=100000, leverage=1, lot_size=1, slippage=0.0001, rpt = 1):
    
    trade_book = []
    in_trade_long = False
    in_trade_short = False
    signal_initial_sl_long = 0
    signal_initial_sl_short = 0
    cumulative_roi = 0
    max_drawdown = 0
    peak_roi = 0

    # print(df.tail(50).to_string())

    # for i in range(1, len(df)):
    #     points = 0
        
    #     current_candle_open = df.iloc[i]["open"]
    #     current_candle_high = df.iloc[i]["high"]
    #     current_candle_low = df.iloc[i]["low"]
    #     current_candle_close = df.iloc[i]["close"]

    #     previous_candle_open = df.iloc[i-1]["open"]
    #     previous_candle_high = df.iloc[i-1]["high"]
    #     previous_candle_low = df.iloc[i-1]["low"]
    #     previous_candle_close = df.iloc[i-1]["close"]

    #     if not in_trade_long:
    #         if (df.iloc[i-1]['signal'] == 1) and current_candle_high >= previous_candle_high and current_candle_open <= previous_candle_high:
    #             # Entry Triggered on SPOT
    #             entry_price_long = previous_candle_high
    #             initial_sl_long = entry_price_long * (1 - (sl_pct / 100))
    #             # initial_sl_long = df.iloc[max(0, i-n):i]['low'].min()
    #             signal_generation_time_long = df.iloc[i-1]['datetime']
    #             entry_time_long = df.iloc[i]['datetime']
    #             in_trade_long = True
                
    #             # OPTIONS Segment
    #             spot_df = spot_dataframe[(spot_dataframe['datetime'] >= entry_time_long][:15]
    #             first_match = spot_df[spot_df['high'] >= trade["Entry Price"]].iloc[0]    # First row of breach
    #             atm_strike = int(round(entry_price_long / 100) * 100)
    #             asset_class = 'P'

    #     if in_trade_long:

    #         if current_candle_low <= initial_sl_long:
    #             if current_candle_open > initial_sl_long:
    #                 # Initial SL Hit
    #                 in_trade_long = False
    #                 exit_price_long = initial_sl_long
    #                 exit_time_long = df.iloc[i]['datetime']
    #                 points = exit_price_long - entry_price_long
    #                 remarks = 'ISL Hit'
    #             elif current_candle_open < initial_sl_long  and (df.iloc[i]['datetime'] != entry_time_long):
    #                 # Gap Open SL
    #                 in_trade_long = False
    #                 exit_price_long = current_candle_open
    #                 exit_time_long = df.iloc[i]['datetime']
    #                 points = exit_price_long - entry_price_long
    #                 remarks = 'Gap SL Hit'
    #             else:
    #                 # Initial SL Hit
    #                 in_trade_long = False
    #                 exit_price_long = initial_sl_long
    #                 exit_time_long = df.iloc[i]['datetime']
    #                 points = exit_price_long - entry_price_long
    #                 remarks = 'ISL Hit'
                    
    #         elif df.iloc[i-1]['signal'] == -1 and current_candle_low <= previous_candle_low and previous_candle_low > initial_sl_long:
    #             # MA Cross in Opposite Direction
    #             if current_candle_open >= previous_candle_low:
    #                 in_trade_long = False
    #                 exit_price_long = previous_candle_low
    #                 exit_time_long = df.iloc[i]['datetime']
    #                 points = exit_price_long - entry_price_long
    #                 remarks = 'MA Cross Opp'
    #             else:
    #                 if current_candle_high >= previous_candle_low:
    #                     in_trade_long = False
    #                     exit_price_long = previous_candle_low
    #                     exit_time_long = df.iloc[i]['datetime']
    #                     points = exit_price_long - entry_price_long
    #                     remarks = 'MA Cross Opp'
    #                 else:
    #                     in_trade_long = False
    #                     exit_price_long = current_candle_close
    #                     exit_time_long = df.iloc[i]['datetime']
    #                     points = exit_price_long - entry_price_long
    #                     remarks = 'MA Cross Opp W Gap Exit'

    for i in range(1, len(df)):
        points = 0
        current_time = df.iloc[i]['datetime']
        current_price = df.iloc[i]['close']
        current_candle_high = df.iloc[i]['high']
        current_candle_low = df.iloc[i]['low']
        previous_candle_high = df.iloc[i - 1]['high']
        previous_candle_low = df.iloc[i - 1]['low']

    
        if not in_trade_long:
            if (df.iloc[i - 1]['signal'] == 1) and current_candle_high >= previous_candle_high:
                entry_price_long = previous_candle_high
                initial_sl_long = entry_price_long * (1 - (sl_pct / 100))
                entry_time_long = current_time
                in_trade_long = True
    
                # Find the exact minute of breach in spot_df
                spot_df_filtered = spot_df[(spot_df['datetime'] >= entry_time_long) & (spot_df['high'] >= entry_price_long)]
                print(spot_df_filtered)
                if not spot_df_filtered.empty:
                    exact_breach_time = spot_df_filtered.iloc[0]['datetime']
                else:
                    exact_breach_time = entry_time_long
    
                atm_strike = int(round(entry_price_long / 100) * 100)
                # Find the nearest expiry date greater than or equal to entry_time_long
                current_expiry = min([d for d in expiries if d >= entry_time_long], default=None)
    
                trade_book.append({
                    "Entry Time": exact_breach_time,
                    "Entry Price": entry_price_long,
                    "Initial SL": initial_sl_long,
                    "ATM Strike": atm_strike,
                    "Expiry": current_expiry
                })
    
        if in_trade_long:
            if current_candle_low <= initial_sl_long:
                if current_candle_open > initial_sl_long:
                    exit_price_long = initial_sl_long
                    exit_time_long = current_time
                    points = exit_price_long - entry_price_long
                    remarks = 'ISL Hit'
                elif current_candle_open < initial_sl_long and (current_time != entry_time_long):
                    exit_price_long = current_candle_open
                    exit_time_long = current_time
                    points = exit_price_long - entry_price_long
                    remarks = 'Gap SL Hit'
                else:
                    exit_price_long = initial_sl_long
                    exit_time_long = current_time
                    points = exit_price_long - entry_price_long
                    remarks = 'ISL Hit'
    
                in_trade_long = False
                trade_book[-1].update({
                    "Exit Time": exit_time_long,
                    "Exit Price": exit_price_long,
                    "Points Captured": points,
                    "Remarks": remarks
                })
    
            elif df.iloc[i - 1]['signal'] == -1 and current_candle_low <= previous_candle_low and previous_candle_low > initial_sl_long:
                if current_candle_open >= previous_candle_low:
                    exit_price_long = previous_candle_low
                elif current_candle_high >= previous_candle_low:
                    exit_price_long = previous_candle_low
                else:
                    exit_price_long = current_price
    
                exit_time_long = current_time
                points = exit_price_long - entry_price_long
                remarks = 'MA Cross Opp' if current_candle_high >= previous_candle_low else 'MA Cross Opp W Gap Exit'
    
                in_trade_long = False
                trade_book[-1].update({
                    "Exit Time": exit_time_long,
                    "Exit Price": exit_price_long,
                    "Points Captured": points,
                    "Remarks": remarks
                })
    
    # Filter expiries that fall within the trade's duration
    # trade_start_time = trade_book[0]["Entry Time"] if trade_book else None
    # trade_end_time = trade_book[-1]["Exit Time"] if trade_book else None
    # if trade_start_time and trade_end_time:
    #     relevant_expiries = [d for d in expiries if trade_start_time <= d <= trade_end_time]
    #     print("Relevant Expiries:", relevant_expiries)


        #     if not in_trade_long and points:
        #         # Exit Found
        #         # qty = int(round((portfolio * leverage / entry_price_long) / lot_size)) * lot_size
        #         qty = int(round((portfolio * rpt / 100) / abs(entry_price_long - initial_sl_long)) / lot_size) * lot_size
        #         slippage_ = slippage * (entry_price_long + exit_price_long)
        #         final_points = points - slippage_
        #         pnl = final_points * qty
        #         roi = (pnl / portfolio) * 100
                
        #         trade_book.append({
        #             "Trade Type": "LONG",
        #             "Entry Time": entry_time_long,
        #             "Entry Price": entry_price_long,
        #             "Initial SL": initial_sl_long,
        #             "Exit Time": exit_time_long,
        #             "Exit Price": exit_price_long,
        #             "Points Captured": points,
        #             "Slippage": slippage_,
        #             "Qty": qty,
        #             "Final Points": final_points,
        #             "PnL": pnl,
        #             "ROI%": roi,
        #             "Trade Duration": exit_time_long - entry_time_long,
        #             "Remarks": remarks,
        #         })

        #         remarks = ""
        #         points = 0

                

        # if not in_trade_short:
        #     if (df.iloc[i-1]['signal'] == -1) and current_candle_low <= previous_candle_low and current_candle_open >= previous_candle_low:
        #         # Entry Triggered for Short
        #         entry_price_short = previous_candle_low
        #         initial_sl_short = entry_price_short * (1 + (sl_pct / 100))
        #         # initial_sl_short = df.iloc[max(0, i-n):i]['high'].max()
        #         entry_time_short = df.iloc[i]['datetime']
        #         in_trade_short = True

        # if in_trade_short:
            
        #     if current_candle_high >= initial_sl_short:
        #         if current_candle_open < initial_sl_short:
        #             # Initial SL Hit for Short
        #             in_trade_short = False
        #             exit_price_short = initial_sl_short
        #             exit_time_short = df.iloc[i]['datetime']
        #             points = entry_price_short - exit_price_short
        #             remarks = 'ISL Hit'
        #         elif current_candle_open < initial_sl_short and (df.iloc[i]['datetime'] != entry_time_short):
        #             # Initial SL Hit for Short
        #             in_trade_short = False
        #             exit_price_short = current_candle_open
        #             exit_time_short = df.iloc[i]['datetime']
        #             points = entry_price_short - exit_price_short
        #             remarks = 'Gap SL Hit'
        #         else:
        #             # Initial SL Hit for Short
        #             in_trade_short = False
        #             exit_price_short = initial_sl_short
        #             exit_time_short = df.iloc[i]['datetime']
        #             points = entry_price_short - exit_price_short
        #             remarks = 'ISL Hit'
                    
        #     elif df.iloc[i-1]['signal'] == 1 and current_candle_high >= previous_candle_high and previous_candle_high < initial_sl_short:
        #         # MA Cross in Opposite Direction for Short
        #         if current_candle_open <= previous_candle_high:
        #             in_trade_short = False
        #             exit_price_short = previous_candle_high
        #             exit_time_short = df.iloc[i]['datetime']
        #             points = entry_price_short - exit_price_short
        #             remarks = 'MA Cross Opp'
        #         else:
        #             if current_candle_low <= previous_candle_high:
        #                 in_trade_short = False
        #                 exit_price_short = previous_candle_high
        #                 exit_time_short = df.iloc[i]['datetime']
        #                 points = entry_price_short - exit_price_short
        #                 remarks = 'MA Cross Opp'
        #             else:
        #                 in_trade_short = False
        #                 exit_price_short = current_candle_close
        #                 exit_time_short = df.iloc[i]['datetime']
        #                 points = entry_price_short - exit_price_short
        #                 remarks = 'MA Cross Opp W Gap Exit'

        #     if not in_trade_short and points:
        #         # Exit Found
        #         # qty = int(round((portfolio * leverage / entry_price_short) / lot_size)) * lot_size
        #         qty = int(round((portfolio * rpt / 100) / abs(entry_price_short - initial_sl_short)) / lot_size) * lot_size
        #         slippage_ = slippage * (entry_price_short + exit_price_short)
        #         final_points = points - slippage_
        #         pnl = final_points * qty
        #         roi = (pnl / portfolio) * 100

        #         trade_book.append({
        #             "Trade Type": "SHORT",
        #             "Entry Time": entry_time_short,
        #             "Entry Price": entry_price_short,
        #             "Initial SL": initial_sl_short,
        #             "Exit Time": exit_time_short,
        #             "Exit Price": exit_price_short,
        #             "Points Captured": points,
        #             "Slippage": slippage_,
        #             "Qty": qty,
        #             "Final Points": final_points,
        #             "PnL": pnl,
        #             "ROI%": roi,
        #             "Trade Duration": exit_time_short - entry_time_short,
        #             "Remarks": remarks,
        #         })

        #         remarks = ""
        #         points = 0

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [75]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [76]:
PORTFOLIO = 50_00_000
LEVERAGE_ = 5
LOT_SIZE_ = 30
SLIPPAGE_ = 0.0001
RPT_ = 2

In [77]:
stats_dictionary = {}

slow_ma = 18
fast_ma = 6
# trailing_ma = 80
sl_pct = 0.75

df = ma_crossover_logic(bnf_pandas, fast_ma, slow_ma)

tb = execute(df, spot_df, sl_pct, 10, PORTFOLIO, LEVERAGE_, LOT_SIZE_, SLIPPAGE_, RPT_)
variation = f'MA1 : {slow_ma}, MA2 : {fast_ma}, SL : {sl_pct}%'

                  datetime       open       high        low      close
300    2017-01-02 14:15:00 18018.8500 18048.9000 18018.6500 18046.4500
301    2017-01-02 14:16:00 18042.2500 18050.7000 18040.0500 18049.4500
302    2017-01-02 14:17:00 18048.8500 18052.8500 18035.7500 18039.5500
303    2017-01-02 14:18:00 18038.1500 18043.9000 18031.1500 18033.0500
304    2017-01-02 14:19:00 18034.4500 18037.2500 18026.3500 18031.5000
...                    ...        ...        ...        ...        ...
750523 2025-02-07 12:53:00 50118.7500 50132.3500 50109.0500 50111.3500
750524 2025-02-07 12:54:00 50111.9000 50115.8000 50089.3000 50089.5500
750525 2025-02-07 12:55:00 50090.2500 50110.3000 50081.3500 50081.3500
750526 2025-02-07 12:56:00 50083.5000 50128.4000 50080.4000 50113.3500
750527 2025-02-07 12:57:00 50109.5500 50121.4000 50095.4500 50104.8000

[746246 rows x 5 columns]


NameError: name 'expiries' is not defined

In [26]:
tb['Trade Year'] = tb['Entry Time'].dt.year
tb = tb.sort_values(by="Entry Time")
stats = generate_stats(tb, variation)

In [27]:
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                      Variation
2017      36.8605          315  35.2381                1.8055             -0.8017     -17.4352       2.1141  MA1 : 18, MA2 : 6, SL : 0.75%
2018     100.5037          306  39.5425                2.3533             -0.9959     -18.9632       5.2999  MA1 : 18, MA2 : 6, SL : 0.75%
2019     100.9812          309  38.5113                2.6393             -1.1216     -27.4912       3.6732  MA1 : 18, MA2 : 6, SL : 0.75%
2020     307.6505          366  35.5191                5.4629             -1.7056     -24.5675      12.5227  MA1 : 18, MA2 : 6, SL : 0.75%
2021      95.7113          343  35.8601                3.0862             -1.2904     -39.4799       2.4243  MA1 : 18, MA2 : 6, SL : 0.75%
2022      51.6445          335  36.7164                2.7081             -1.3276     -24.6768       2.0928  MA1 : 18, MA2 : 6, SL : 0.75%
2023      43.6569          

In [23]:
tb.tail(250)

,Trade Type,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Points Captured,Slippage,Qty,Final Points,PnL,ROI%,Trade Duration,Remarks,Trade Year
2208,SHORT,2024-04-08 14:45:00,48574.6500,48938.9599,2024-04-08 15:00:00,48613.5000,-38.8500,9.7188,270,-48.5688,-13113.5800,-0.2623,0 days 00:15:00,MA Cross Opp,2024
2209,LONG,2024-04-08 15:00:00,48613.5000,48248.8988,2024-04-09 14:00:00,48656.4500,42.9500,9.7270,270,33.2230,8970.2113,0.1794,0 days 23:00:00,MA Cross Opp,2024
2210,SHORT,2024-04-09 14:00:00,48656.4500,49021.3734,2024-04-10 10:15:00,48887.7500,-231.3000,9.7544,270,-241.0544,-65084.6934,-1.3017,0 days 20:15:00,MA Cross Opp,2024
2211,LONG,2024-04-10 10:15:00,48887.7500,48521.0919,2024-04-12 10:00:00,48665.1500,-222.6000,9.7553,270,-232.3553,-62735.9283,-1.2547,1 days 23:45:00,MA Cross Opp,2024
2212,SHORT,2024-04-12 10:00:00,48665.1500,49030.1386,2024-04-16 15:15:00,47525.3500,1139.8000,9.6190,270,1130.1810,305148.8565,6.1030,4 days 05:15:00,MA Cross Opp,2024
2213,LONG,2024-04-16 15:15:00,47525.3500,47168.9099,2024-04-18 13:30:00,47168.9099,-356.4401,9.4694,270,-365.9096,-98795.5788,-1.9759,1 days 22:15:00,ISL Hit,2024
2214,SHORT,2024-04-18 13:30:00,47267.0500,47621.5529,2024-04-19 12:45:00,47243.2500,23.8000,9.4510,270,14.3490,3874.2219,0.0775,0 days 23:15:00,MA Cross Opp,2024
2215,LONG,2024-04-19 12:45:00,47243.2500,46888.9256,2024-04-23 14:00:00,47969.2500,726.0000,9.5213,270,716.4787,193449.2625,3.8690,4 days 01:15:00,MA Cross Opp,2024
2216,SHORT,2024-04-23 14:00:00,47969.2500,48329.0194,2024-04-24 09:45:00,48112.0500,-142.8000,9.6081,270,-152.4081,-41150.1951,-0.8230,0 days 19:45:00,MA Cross Opp,2024
2217,LONG,2024-04-24 09:45:00,48112.0500,47751.2096,2024-04-24 15:00:00,48153.5500,41.5000,9.6266,270,31.8734,8605.8288,0.1721,0 days 05:15:00,MA Cross Opp,2024


In [24]:
tb['Trade Duration'].mean()

Timedelta('1 days 04:12:56.973148901')

In [19]:
# tb.to_csv('full_ma_cross_4_20_MAs_15m_TF.csv')

In [ ]:
sl_pct_range = [0.5, 0.75, 1, 1.25, 1.5]
stats_dictionary = {}

PORTFOLIO = 1_00_00_000
LEVERAGE_ = 5
LOT_SIZE_ = 30
SLIPPAGE_ = 0.0001

for i in range(4, 41, 2):
    for j in range(6, 81, 2):
        for sl in sl_pct_range:
            if i < j and ((j-i) <= 16):
                variation = f'MA1 : {i}, MA2 : {j}, SL : {sl}%'
                print(variation)
                df = ma_crossover_logic(bnf_pandas, i, j)
                tb = execute(df, sl, 1, PORTFOLIO, LEVERAGE_, LOT_SIZE_, SLIPPAGE_, 3)
                if len(tb) > 0:
                    tb['Trade Year'] = tb['Entry Time'].dt.year
                    tb = tb.sort_values(by="Entry Time")
                    stats = generate_stats(tb, variation)

                    for overall_roi_dd_ratio, stats_df in stats.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 15:
                            print(stats_df.to_string())
                            stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 4, MA2 : 6, SL : 0.5%
MA1 : 4, MA2 : 6, SL : 0.75%
MA1 : 4, MA2 : 6, SL : 1%
